In [1]:
import json
import requests

import dlt
from dlt.destinations import qdrant

from qdrant_client import QdrantClient
client = QdrantClient("http://localhost:6333")

In [2]:
qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [3]:
print(f'dlt version: {dlt.__version__}')

dlt version: 1.12.3


In [4]:
@dlt.resource
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

Run started at 2025-07-14 03:24:32.236739+00:00 and COMPLETED in 10.38 seconds with 4 steps.
Step extract COMPLETED in 1.21 seconds.

Load package 1752463473.009967 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.08 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)

Load package 1752463473.009967 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 8.34 seconds.
Pipeline zoomcamp_pipeline load step completed in 8.33 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /home/rajinder-mavi/code/nlp_sandbox/llm_zoomcamp/db.qdrant location to store data
Load package 1752463473.009967 is LOADED and contains no failed jobs

Step run COMPLETED in 10.38 seconds.
Pipeline zoomcamp_pipeline load step completed in 8.33 seconds
1 load package(s) were loaded to destination qdrant and int

In [5]:
with open('db.qdrant/meta.json', 'r') as file:
    dlt_meta = json.load(file)

In [6]:
dlt_meta['collections']['zoomcamp_tagged_data']['vectors']

{'fast-bge-small-en': {'size': 384,
  'distance': 'Cosine',
  'hnsw_config': None,
  'quantization_config': None,
  'on_disk': None,
  'datatype': None,
  'multivector_config': None}}